# Analysing Chinook Database

We'll anaylise this database called Chinnok, representing the sales system of a fictional digital music shop.

As stated on the source on [github](https://github.com/lerocha/chinook-database):

> Media related data was created using real data from an iTunes Library [...] Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.

Here's the schema diagram:

![Chinook Database Schema](chinook-schema.svg "Chinook Database Schema")

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Best Genres in the USA

In [38]:
%%sql
DROP VIEW IF EXISTS USA_sales;

CREATE VIEW USA_sales AS
SELECT genre.name genre, invoice_line.quantity
  FROM customer
 INNER JOIN invoice ON invoice.customer_id = customer.customer_id
 INNER JOIN invoice_line ON invoice_line.invoice_id = invoice.invoice_id
 INNER JOIN track ON track.track_id = invoice_line.track_id
 INNER JOIN genre ON genre.genre_id = track.genre_id
 WHERE country = "USA";

Done.
Done.


[]

In [40]:
%%sql

SELECT genre, 
       SUM(quantity) total_sold,
       ROUND(CAST(SUM(quantity) AS FLOAT)/(
                                           SELECT SUM(quantity)
                                             FROM USA_sales
                                           ),2) "percentage sold"
  FROM USA_sales
 GROUP BY genre
 ORDER BY total_sold DESC
 LIMIT 10;


Done.


genre,total_sold,percentage sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


### Observations
Looking at the 4 genres they're proposing and the most selling genres in the USA, we should purchase Red Tone (Punk, 12%), Slim Jim Bites (Blues, 3%), and Meteor and the Girls (Pop, 2%)

## Employee performance

In [74]:
%%sql
DROP VIEW sales;

CREATE VIEW sales AS
SELECT *
  FROM employee
 INNER JOIN customer ON customer.support_rep_id = employee.employee_id
 INNER JOIN invoice ON invoice.customer_id = customer.customer_id


Done.
Done.


[]

In [96]:
%%sql
SELECT sales.first_name || " " || sales.last_name employee_name,
       COUNT(DISTINCT customer_id) customers, 
       round(SUM(total)) total_sales_value,
       ROUND(SUM(total)/COUNT(DISTINCT customer_id)) average_spent,
       birthdate,
       hire_date
FROM sales
group by employee_id
limit 10

Done.


employee_name,customers,total_sales_value,average_spent,birthdate,hire_date
Jane Peacock,21,1732.0,82.0,1973-08-29 00:00:00,2017-04-01 00:00:00
Margaret Park,20,1584.0,79.0,1947-09-19 00:00:00,2017-05-03 00:00:00
Steve Johnson,18,1394.0,77.0,1965-03-03 00:00:00,2017-10-17 00:00:00


### Observations
We can see some differences between the three sales agents, but it's interesting to notice that better performing agents have more customers and they tend to spend more. Also, Jane is the youngest but the most experienced one.

## Sales by country

In [98]:
%%sql
SELECT customer.customer_id,
       country,
       round(SUM(total)) total_spent,
       round(SUM(total)/COUNT(total)) average_order_value
  FROM customer
 INNER JOIN invoice on invoice.customer_id = customer.customer_id
 GROUP BY customer.customer_id

Done.


customer_id,country,total_spent,average_order_value
1,Brazil,109.0,8.0
2,Germany,82.0,7.0
3,Canada,100.0,11.0
4,Norway,72.0,8.0
5,Czech Republic,145.0,8.0
6,Czech Republic,129.0,11.0
7,Austria,69.0,8.0
8,Belgium,60.0,9.0
9,Denmark,38.0,4.0
10,Brazil,60.0,5.0


In [171]:
%%sql

WITH country_customers 
AS
(
SELECT country,
       COUNT(DISTINCT customer.customer_id) customers,
       SUM(total) total_spent,
       SUM(total)/COUNT(total) average_order_value
  FROM customer
 INNER JOIN invoice ON invoice.customer_id = customer.customer_id
 GROUP BY country
)

SELECT cc.country2 AS country,
       ROUND(SUM(customers)) customers,
       ROUND(SUM(total_spent)) total_spent,
       ROUND(AVG(average_order_value)) average_order_value,
       ROUND(SUM(total_spent)/SUM(customers)) average_total_per_customer

FROM (
     SELECT country_customers.*,
           CASE WHEN customers = 1 
                THEN 1
                ELSE 0
           END AS sort,
           CASE WHEN customers = 1
                THEN 'Other'
                ELSE country
           END AS country2
    FROM country_customers
    ) cc

GROUP BY cc.country2
ORDER BY sort ASC, customers DESC
    

Done.


country,customers,total_spent,average_order_value,average_total_per_customer
USA,13.0,1040.0,8.0,80.0
Canada,8.0,536.0,7.0,67.0
Brazil,5.0,428.0,7.0,86.0
France,5.0,389.0,8.0,78.0
Germany,4.0,335.0,8.0,84.0
United Kingdom,3.0,246.0,9.0,82.0
Czech Republic,2.0,273.0,9.0,137.0
India,2.0,183.0,9.0,92.0
Portugal,2.0,185.0,6.0,93.0
Other,15.0,1095.0,7.0,73.0


### Observations
- USA has the highest number of customers
- customers from UK and Czech Republic tend to spend the most per each average order
- It's interesting to notice that Czech Republic, India, and Portugal has only two customers but with a high lifetime spending.

## Albums vs Individual Tracks

In [34]:
%%sql

WITH invoice_album AS
(
    SELECT invoice_id, MIN(track_id) first_track, album_id
     FROM 
        (
            SELECT il.invoice_id, il.track_id, album_id
              FROM invoice_line il
             INNER JOIN track ON track.track_id = il.track_id
        )
     GROUP BY invoice_id
)

SELECT 
        CASE 
            WHEN
                (SELECT track_id
                   FROM invoice_line
                  WHERE invoice_id = ia.invoice_id
            EXCEPT
                SELECT track.track_id
                  FROM track
                WHERE album_id = ia.album_id
                ) IS NULL
            AND 

            (SELECT track.track_id
                  FROM track
                WHERE album_id = ia.album_id
            EXCEPT
                SELECT track_id
                   FROM invoice_line
                  WHERE invoice_id = ia.invoice_id
            ) IS NULL
            THEN 'yes'
            ELSE 'no'
        END AS album_purchase,
        COUNT(invoice_id) purchases,
        ROUND(CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(invoice_id)
                                       FROM invoice),3)percentage
FROM invoice_album ia
GROUP BY album_purchase

Done.


album_purchase,purchases,percentage
no,500,0.814
yes,114,0.186


### Observations 
More than 81% of purchases are mixted tracks, so a suggestion would be to stop buying whole album and start buying only the most popular tracks from each album from record companies.